In [ ]:

from torch.utils.data import Dataset
from torchvision import transforms
from glob import glob
import os
import random
from PIL import Image
import torch
import numpy as np
SMOOTH = 1e-6
class SemSegData(Dataset):

    def __init__(self, root_dir):
        self.root_dir = root_dir
        
        self._init_dataset()
        
    def _init_dataset(self):

        self.img_list = glob(self.root_dir + '/original_images/*') #creates a list of images
        self.mask_list = glob(self.root_dir + '/label_images_semantic/*')
        
    
    
    def __getitem__(self,index):
        img = self.img_list[index]
        mask = self.mask_list[index]

        img = (torch.tensor(np.array(Image.open(img).resize((128,128)))).permute(2,0,1) )/255 #(128,128,3)->(3,128,128)
        
        
        mask = torch.unsqueeze(torch.tensor(np.array(Image.open(mask).resize((128,128),Image.NEAREST))),0) #128x128 ->1x128x128
        
            
        return img, mask

    def __len__(self):
        return len(self.mask_list)


In [ ]:
obj = SemSegData(root_dir = '/content/drive/MyDrive/Colab Notebooks/semantic_drone_dataset')
obj[0]

IndexError: ignored

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

In [ ]:
from torchsummary import summary

In [ ]:
model = UNET()
#model.to("cuda")

In [ ]:
summary(model, (3,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,728
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
            Conv2d-4           [-1, 64, 64, 64]          36,864
       BatchNorm2d-5           [-1, 64, 64, 64]             128
              ReLU-6           [-1, 64, 64, 64]               0
        DoubleConv-7           [-1, 64, 64, 64]               0
         MaxPool2d-8           [-1, 64, 32, 32]               0
            Conv2d-9          [-1, 128, 32, 32]          73,728
      BatchNorm2d-10          [-1, 128, 32, 32]             256
             ReLU-11          [-1, 128, 32, 32]               0
           Conv2d-12          [-1, 128, 32, 32]         147,456
      BatchNorm2d-13          [-1, 128, 32, 32]             256
             ReLU-14          [-1, 128,

In [ ]:
"""def test():
    x = torch.randn((3, 1, 161, 161))
    model = UNET(in_channels=1, out_channels=1)
    preds = model(x)
    assert preds.shape == x.shape

#if __name__ == "__main__":
test()"""

In [ ]:
import torch.optim as optim
import torchvision
import os
from torch.utils.data import DataLoader, random_split
#import datasets
import random
import numpy as np 

def train():

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_epochs = 200
    batch_size = 8
    learning_rate = 0.001

    
    def soft_dice_loss(y_true, y_pred, epsilon=1e-6): 
        
        mask = torch.zeros((y_true.size(0),24,128,128)).to(device)
        outs = torch.softmax(y_pred,1)
        mask.permute(0,2,3,1).contiguous().view(-1,24)[torch.arange(y_true.size(0)*128*128),y_true.view(-1)] = 1
        
        numerator = 2. * (outs *mask).sum((2,3))
        denominator = torch.sum(torch.square(outs) + torch.square(mask),(2,3))
        
        return torch.mean(1 - torch.mean((numerator + epsilon) / (denominator + epsilon),1)) 


    dataset = SemSegData(root_dir='/content/drive/MyDrive/Colab Notebooks/semantic_drone_dataset')
    data_len = len(dataset)

    trainset, valset = random_split(dataset, [int(0.8*data_len), (data_len - int(0.8*data_len))])

    trainloader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle = True)
    valloader = DataLoader(dataset=valset, batch_size=batch_size, shuffle = False)

    model = UNET().to(device)
    
    criterion = nn.CrossEntropyLoss()
    best = float("inf")
    train_step = 0
    val_step = 0
    lr_lbmd = lambda e: max(0.7**(e//20), 0.00001/0.001)
    
    optimiser = optim.Adam(model.parameters(), lr=learning_rate)
    
    
   
        
    lr_scheduler = optim.lr_scheduler.LambdaLR(optimiser, lr_lbmd)

    for epoch in range(num_epochs):
        print(f"Epoch: {epoch}")
        print("---------------")
        model.train()
        loss_cntr = []
        torch.cuda.empty_cache()

        for idx, (img, mask) in enumerate(trainloader):
            
            img, mask = img.to(device), mask.to(device)
            mask = mask.long()
            
            outputs = model(img)
            
            loss = criterion(outputs.permute(0,2,3,1).contiguous().view(-1,24),mask.view(-1,))
            loss2 = soft_dice_loss(mask,outputs)
            t_loss = loss + loss2
            optimiser.zero_grad()
            t_loss.backward(retain_graph = False)
            optimiser.step()
            loss_cntr.append(loss.item())
            train_step += 1

            datasets.progress_bar(progress=indx/len(trainloader), status=f"loss: {round(np.mean(loss_cntr), 4)}")
        datasets.progress_bar(progress=1, status=f"loss: {round(np.mean(loss_cntr), 4)}")
        print("\n")

        loss_cntr = []
        model.eval()
        
        torch.cuda.empty_cache()
        with torch.no_grad():
            for idx, (img2, mask2) in enumerate(valloader):
                
                img2, mask2 = img2.to(device), mask2.to(device)
                out2 = model(img2)
                mask2 = mask2.long()
                loss = criterion(out2.permute(0,2,3,1).contiguous().view(-1,24),mask2.view(-1,))
                loss2 = soft_dice_loss(mask2,out2)
            
                loss = loss + loss2        
                
                loss_cntr.append(loss.item())
                val_step += 1

                datasets.progress_bar(progress=indx/len(valloader), status=f"loss: {round(np.mean(loss_cntr), 4)}")
            datasets.progress_bar(progress=1, status=f"loss: {round(np.mean(loss_cntr), 4)}")

        print("\n")
        
if __name__ == "__main__":
    train()            

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
